In [1]:
import cv2
import pandas as pd
import os
import numpy as np

# --- Cấu hình (ĐÃ CẬP NHẬT) ---
BASE_VIDEO_PATH = "DATA/RawVideo"  # <-- ĐÃ THAY ĐỔI
CLIP_OUTPUT_PATH = "DATA/data_clips/test" # Thư mục lưu clip test đã cắt
ANNOTATION_FILE = "DATA/cleaned_annotations.csv" # File CSV đã làm sạch từ bước trước



In [2]:
# --- (MỚI) Xây dựng bộ đệm (cache) tìm kiếm file ---
# Quét toàn bộ thư mục BASE_VIDEO_PATH và các thư mục con
print(f"Đang quét thư mục video gốc: {BASE_VIDEO_PATH}...")
video_path_cache = {} # Dict để tra cứu: "video_name.mp4" -> "full/path/to/video_name.mp4"

for root, dirs, files in os.walk(BASE_VIDEO_PATH):
    for file in files:
        if file.endswith(('.mp4', '.avi', '.mkv', '.x264')): # Hỗ trợ các định dạng video
            video_path_cache[file] = os.path.join(root, file)

print(f"Đã tìm thấy {len(video_path_cache)} video trong {BASE_VIDEO_PATH}.")
# ----------------------------------------------------



Đang quét thư mục video gốc: DATA/RawVideo...
Đã tìm thấy 1900 video trong DATA/RawVideo.


In [3]:
# Đảm bảo các thư mục đầu ra tồn tại
if not os.path.exists(CLIP_OUTPUT_PATH):
    os.makedirs(CLIP_OUTPUT_PATH)

# --- Hàm cắt video (ĐÃ CẬP NHẬT) ---
def trim_video_clip(source_video_name, output_clip_name, label, start_frame, end_frame):
    """
    Cắt một video clip từ frame_start đến frame_end và lưu nó.
    """
    
    # (CẬP NHẬT) Tìm đường dẫn video từ cache
    if source_video_name not in video_path_cache:
        print(f"CẢNH BÁO: Không tìm thấy video '{source_video_name}' trong {BASE_VIDEO_PATH} hoặc các thư mục con.")
        return
    
    source_video_full_path = video_path_cache[source_video_name] # <-- Lấy từ cache
    
    # Tạo thư mục cho nhãn nếu chưa có (ví dụ: data_clips/test/Arson)
    label_dir = os.path.join(CLIP_OUTPUT_PATH, label)
    if not os.path.exists(label_dir):
        os.makedirs(label_dir)
        
    output_clip_full_path = os.path.join(label_dir, output_clip_name)

    # 1. Mở video gốc
    cap = cv2.VideoCapture(source_video_full_path)
    if not cap.isOpened():
        print(f"LỖI: Không thể mở video gốc: {source_video_full_path}")
        return

    # Lấy thông tin video (FPS, kích thước frame)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # 2. Tạo đối tượng VideoWriter để lưu clip mới
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Codec
    writer = cv2.VideoWriter(output_clip_full_path, fourcc, fps, (frame_width, frame_height))

    print(f"Đang xử lý: {output_clip_full_path}...")

    # 3. Đặt vị trí frame bắt đầu
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    
    current_frame = start_frame
    
    # 4. Lặp và ghi đè các frame
    while current_frame <= end_frame:
        ret, frame = cap.read()
        if not ret:
            break
        writer.write(frame)
        current_frame += 1

    # 5. Giải phóng tài nguyên
    cap.release()
    writer.release()



In [4]:
# --- Hàm xử lý cho video 'Normal' (ĐÃ CẬP NHẬT) ---
def copy_normal_video(source_video_name, label):
    """
    Cắt video Normal thành các đoạn 5 giây.
    """
    
    # (CẬP NHẬT) Tìm đường dẫn video từ cache
    if source_video_name not in video_path_cache:
        print(f"CẢNH BÁO: Không tìm thấy video '{source_video_name}' trong {BASE_VIDEO_PATH} hoặc các thư mục con.")
        return
            
    source_video_full_path = video_path_cache[source_video_name] # <-- Lấy từ cache
    
    label_dir = os.path.join(CLIP_OUTPUT_PATH, label)
    if not os.path.exists(label_dir):
        os.makedirs(label_dir)

    cap = cv2.VideoCapture(source_video_full_path)
    if not cap.isOpened():
        print(f"LỖI: Không thể mở video gốc: {source_video_full_path}")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps == 0: fps = 30 # Giá trị mặc định nếu không đọc được
        
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    clip_duration_frames = int(fps * 5) # Cắt thành các clip 5 giây
    
    clip_count = 0
    for start_frame in range(0, total_frames, clip_duration_frames):
        end_frame = min(start_frame + clip_duration_frames - 1, total_frames - 1)
        
        # Bỏ qua nếu clip quá ngắn
        if (end_frame - start_frame) < (fps * 1): # Bỏ qua clip dưới 1 giây
            continue
            
        clip_count += 1
        output_clip_name = f"{source_video_name.split('.')[0]}_clip{clip_count}.mp4"
        
        # Tái sử dụng hàm trim (bây giờ đã an toàn vì nó tra cứu cache)
        trim_video_clip(source_video_name, output_clip_name, label, start_frame, end_frame)




In [5]:
# --- Vòng lặp chính (Không thay đổi) ---
print("Bắt đầu quá trình chuẩn bị dữ liệu Test...")
try:
    df = pd.read_csv(ANNOTATION_FILE)
except FileNotFoundError:
    print(f"LỖI: Không tìm thấy file '{ANNOTATION_FILE}'. Bạn đã chạy script để tạo file này chưa?")
    exit()

for index, row in df.iterrows():
    if row['start_frame'] == -1 and row['label'] == 'Normal':
        # Xử lý video Normal
        copy_normal_video(row['source_video'], row['label'])
    elif row['start_frame'] != -1:
        # Xử lý video bất thường
        trim_video_clip(
            row['source_video'],
            row['output_name'],
            row['label'],
            int(row['start_frame']),
            int(row['end_frame'])
        )

print(f"Hoàn thành chuẩn bị dữ liệu Test! Dữ liệu đã được lưu tại: {CLIP_OUTPUT_PATH}")

Bắt đầu quá trình chuẩn bị dữ liệu Test...
Đang xử lý: DATA/data_clips/test\Abuse\Abuse028_x264_clip1.mp4...
Đang xử lý: DATA/data_clips/test\Abuse\Abuse030_x264_clip1.mp4...
Đang xử lý: DATA/data_clips/test\Arrest\Arrest001_x264_clip1.mp4...
Đang xử lý: DATA/data_clips/test\Arrest\Arrest007_x264_clip1.mp4...
Đang xử lý: DATA/data_clips/test\Arrest\Arrest024_x264_clip1.mp4...
Đang xử lý: DATA/data_clips/test\Arrest\Arrest030_x264_clip1.mp4...
Đang xử lý: DATA/data_clips/test\Arrest\Arrest039_x264_clip1.mp4...
Đang xử lý: DATA/data_clips/test\Arson\Arson007_x264_clip1.mp4...
Đang xử lý: DATA/data_clips/test\Arson\Arson009_x264_clip1.mp4...
Đang xử lý: DATA/data_clips/test\Arson\Arson010_x264_clip1.mp4...
Đang xử lý: DATA/data_clips/test\Arson\Arson011_x264_clip1.mp4...
Đang xử lý: DATA/data_clips/test\Arson\Arson011_x264_clip2.mp4...
Đang xử lý: DATA/data_clips/test\Arson\Arson016_x264_clip1.mp4...
Đang xử lý: DATA/data_clips/test\Arson\Arson018_x264_clip1.mp4...
Đang xử lý: DATA/data_c